In [15]:
from new_classes import *
import numpy as np
import pandas as pd
import dill
import os

In [ ]:
for day in range(1, 8):
    df = pd.DataFrame(np.load('day_'+str(day)+'.npy'), columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
           'dropoff_latitude', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
           'passenger_count', 'fare_amount', 'trip_distance', 'day'],
          dtype='object')

    df_sorted = df.sort_values(by=['tpep_pickup_datetime']).set_index(np.arange(len(df)))

    Map = gridMap()
    ride_list = []
    # effective_speed_list = []
    for index, row in df_sorted.iterrows():
        ride = Ride(*([index] + list(row.values) + [Map]))
        if not (ride.ti >= ride.tf) and not (ride.xi == ride.xf).all():
            ride_list.append(ride)
    #         effective_speed_list.append(ride.effective_speed)

    dill.dump(ride_list, open('day' + str(day) + '_ride_list.pkd', 'wb'))


In [ ]:
for day in range(1,8):
    filename = 'day' + str(day) + '_ride_list.pkd' 
    ride_list = dill.load(open(filename, 'rb'))
    Map = gridMap()
    for ride in ride_list:
        ride.gridMap = Map
    
    Map.folder_name('./day' + str(day) + '/')
    for ride in ride_list:
        time = ride.ti
        ride.request_vehicle()
        ride.vehicle.update_vehicle(time)
    Map.clean_map()

In [ ]:
def efficiency_day(i):
    ls = []
    directory = './day' + str(i) + '/'
    for filename in os.listdir(directory):
        share, no_share, number, routes = dill.load(open(directory + filename, 'rb'))
        ls.append([share, no_share, number])

    df = pd.DataFrame(ls, columns=['share', 'no_share', 'number'])
    df = df[df['number'] > 1]

    no_share = df.no_share.values.sum()
    share = df.share.values.sum()
    number = df.number.sum()

    return number, no_share, share, (no_share-share)/number


In [ ]:
efficiency_list = []
for i in range(1, 8):
    efficiency_list.append(efficiency_day(i))

In [ ]:
df = pd.DataFrame(efficiency_list, columns=['number', 'noshare', 'share', 'efficiency'])
(df['noshare'].sum() - df['share'].sum())/df['number'].sum()